In [ ]:
import shutil
import itertools
from pathlib import Path

### When to run this notebook

You received data from a Hyspex survey. Presumably the complete set. You now want to get set up and create a project directory to run processing steps in.

We presume that you obtained these notebooks by cloning a repository, for example https://github.alaska.edu/cwaigl/FireIce_BF_Hyspex , so that the notebook code will be able to access files to be moved in place. 

### Intialize variables


Enter the directory of your HySpex processing project. We presume it doesn't exist yet. 

Make sure to use either forward slashes `/`, or double backslashes `\\` to separate path levels. 

*If you just want to create your directory structure, this is the only variable you need to set.*

In [ ]:
projectdir = Path(r"Z:/fihyper/cwaigl/20210828_KN")

If you also want this notebook to move your raw HySpex files into place (last code section of this notebook), you'll also need to provide the directory where the Hyspex files are stored and the common prefix for the files in this project. 

In [ ]:
prefix = '20210828-KN'
dataarchive = Path(r'Z:\fihyper\Hyspex_Data\2021\BorealForest\20210828-Kenai')

### Create project directory and subdirectories

You don't have to edit anything here if you just want the default setup. Only make changes if you have additional requirements and know what you're doing. Just run each code cell in order. The following list of directories will be created:

In [ ]:
subdirs = [
    '00_aux/Software',
    '00_aux/GIS',
    '00_aux/GIS/Flightline',
    '01_inputs/DEM',
    '01_inputs/sensor',
    '01_inputs/HySpex',
    '01_inputs/boresight',
    '02_intermediate/working',
    '03_products',
    'x0_tmp',
]

Run the following code to create all directories. If they already exist, no error will be raised. If you want an error to be raised, change `exist_ok` to `False`.

In [ ]:
paths = [
    projectdir / Path(subdir) for subdir in subdirs
]

for pth in paths:
    pth.mkdir(parents=True, exist_ok=True)

### Move sensormodel files in place

In the Husky configuration, the cameras are installed 180 degrees rotated. Therefore, `rotated` should be set to `True`. If you are running this script for an unrotated configuration, set the variable to `False`.

In [ ]:
rotated = True
if rotated:
    sensorfilepaths = list(Path('../sensormodel').resolve().glob('*FOVexp_180deg_rotated.txt'))
else:
    sensorfilepaths = list(Path('../sensormodel').resolve().glob('*FOVexp.txt'))
dst = projectdir / Path('01_inputs/sensor')

for fp in sensorfilepaths:
    shutil.copy(fp, dst)

### Move boresight files in place

Boresight calibration needs to be carried out each time that the camera is installed into an aircraft. This generates two new files containing boresight correction parameters, one for the VNIR camera, one for SWIR. 

You may prefer moving boresight files manually into the `01_inputs/boresight` directory. But if you want to use one of the existing boresight correction files, you can uncomment it here and use the following code to achieve that. 

**Check and edit the variables in the next cell before running it.**

In [ ]:
# boresightdate = '20200522'
# boresightdate = '20200625'
boresightdate = '20200912'
boresightDEM = 'IFSAR'
boresightaircraft = 'Husky'

boresightvnirfn = f'{boresightdate}_{boresightaircraft}_VNIR_boresight_{boresightDEM}.gcs'
boresightswirfn = f'{boresightdate}_{boresightaircraft}_SWIR_boresight_{boresightDEM}.gcs'

dst = projectdir / Path('01_inputs/boresight')
for fn in [boresightvnirfn, boresightswirfn]:
    shutil.copy(Path('../boresight', fn), dst)

### Move software in place (NOT)

In this public version of the tool chain, no proprietary software is moved into place automatically. You will need to do this manually or script it with the non-public tools. UAF researchers in need of the software can contact the HyLab team and/or consult the private repo at https://github.alaska.edu/cwaigl/FireIce_BF_Hyspex 

### Copy raw data files to input directory (optional - you can do this manually)

In [ ]:
dst = (projectdir / Path('01_inputs/HySpex')).resolve()

print(f"The directory with the raw HySpex files exists: {dataarchive.is_dir()}")

The directory with the raw HySpex files exists: True


In [ ]:
hyspexfilepaths = list(itertools.chain(dataarchive.glob(f'{prefix}*_raw.hyspex'), 
                   dataarchive.glob(f'{prefix}*_raw.hdr'),
                   dataarchive.glob(f'{prefix}.log'),
                   dataarchive.glob(f'{prefix}*.gps')))

numfiles = len(hyspexfilepaths)
print(f"There are {numfiles} files to be moved")

There are 58 files to be moved


In [ ]:
for ii, fp in enumerate(hyspexfilepaths):
    print(f"Working on copying {fp}")
    try:
        my_abs_path = (dst / Path(fp.parts[-1])).resolve(strict=True)
    except FileNotFoundError:        
        shutil.copy2(fp, dst)
        print(f"Completed copying {fp} ({ii+1}/{numfiles})")
    else:
        print(f"{fp} already exists. Not copying. ({ii+1}/{numfiles})")


Working on copying Z:\fihyper\Hyspex_Data\2021\BorealForest\20210828-Kenai\20210828-KN_01_SWIR_384me_SN3107_FOVx2_raw.hyspex
Completed copying Z:\fihyper\Hyspex_Data\2021\BorealForest\20210828-Kenai\20210828-KN_01_SWIR_384me_SN3107_FOVx2_raw.hyspex (1/58)
Working on copying Z:\fihyper\Hyspex_Data\2021\BorealForest\20210828-Kenai\20210828-KN_01_VNIR_1800_SN00812_FOVx2_raw.hyspex
Completed copying Z:\fihyper\Hyspex_Data\2021\BorealForest\20210828-Kenai\20210828-KN_01_VNIR_1800_SN00812_FOVx2_raw.hyspex (2/58)
Working on copying Z:\fihyper\Hyspex_Data\2021\BorealForest\20210828-Kenai\20210828-KN_02_SWIR_384me_SN3107_FOVx2_raw.hyspex
Completed copying Z:\fihyper\Hyspex_Data\2021\BorealForest\20210828-Kenai\20210828-KN_02_SWIR_384me_SN3107_FOVx2_raw.hyspex (3/58)
Working on copying Z:\fihyper\Hyspex_Data\2021\BorealForest\20210828-Kenai\20210828-KN_02_VNIR_1800_SN00812_FOVx2_raw.hyspex
Completed copying Z:\fihyper\Hyspex_Data\2021\BorealForest\20210828-Kenai\20210828-KN_02_VNIR_1800_SN00812_

### NEXT STEPS

At this stage, go and run HySpex NAV and HySpex RAD on the files. It does not matter which you run first. 

Afterwards go and open the second notebook,[02 Convert HySpex GPS txt to GIS - post NAV.ipynb](02%20Convert%20HySpex%20GPS%20txt%20to%20GIS%20-%20post%20NAV.ipynb)